In [ ]:
from bs4 import BeautifulSoup
import lxml
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup, element
import pandas as pd
import csv
from bs4 import element


In [21]:
def get_book_links(category_id, base_url="https://www.biblio-globus.ru/catalog/category?id="):
    """Извлекает ссылки на книги из категории с указанным id."""
    page_number = 1

    while True:
        url = f"{base_url}{category_id}&page={page_number}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлечение ссылок на книги
        links = soup.find_all('a', class_='img_link')
        if not links:
            print(f"Сканирование category_id {category_id} page_number {page_number} завершено.")
            break  # Выход из цикла, если страница не содержит ссылок

        for link in links:
            book_link = link.get('href')
            if book_link and book_link.startswith('/product/'):
                full_link = f"https://www.biblio-globus.ru{book_link}"
                # Запись в CSV-файл
                with open('book_links.csv', 'a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow([full_link])

        page_number += 1

categories = [226, 227, 241, 242, 248, 250, 251, 6168, 6169, 6170, 6171, 262, 263, 6173, 6174, 6176, 6177, 6178, 6179, 6180, 6181, 6182, 6183, 6184, 6186, 6187, 6188, 6189]  # Добавьте остальные категории по необходимости

# Создание заголовка CSV-файла
with open('book_links.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['book_link'])

# Получение ссылок для каждой категории
for category in categories:
    get_book_links(category)

Сканирование category_id 226 page_number 151 завершено.
Сканирование category_id 227 page_number 413 завершено.
Сканирование category_id 241 page_number 90 завершено.
Сканирование category_id 242 page_number 99 завершено.
Сканирование category_id 248 page_number 5 завершено.
Сканирование category_id 250 page_number 89 завершено.
Сканирование category_id 251 page_number 96 завершено.
Сканирование category_id 6168 page_number 33 завершено.
Сканирование category_id 6169 page_number 34 завершено.
Сканирование category_id 6170 page_number 9 завершено.
Сканирование category_id 6171 page_number 13 завершено.
Сканирование category_id 262 page_number 12 завершено.
Сканирование category_id 263 page_number 16 завершено.
Сканирование category_id 6173 page_number 32 завершено.
Сканирование category_id 6174 page_number 3 завершено.
Сканирование category_id 6176 page_number 4 завершено.
Сканирование category_id 6177 page_number 18 завершено.
Сканирование category_id 6178 page_number 10 завершено.
Ска

In [23]:
def scrape_book_data(url):
    collect = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = {
        'image': soup.find('meta', property="og:image")['content'] if soup.find('meta', property="og:image") else '',
        'author': soup.find('meta', property="og:book:author")['content'] if soup.find('meta', property="og:book:author") else '',
        'title': soup.find('meta', property="og:title")['content'] if soup.find('meta', property="og:title") else '',
        'annotation': soup.find('div', {"class": "collapse", "id": "collapseExample"}) if soup.find('div', {"class": "collapse", "id": "collapseExample"}) else ''
    }

    if content['annotation'] != '' and content['annotation'].children:
        for el in content['annotation'].children:
            if isinstance(el, element.Tag):
                el.decompose()
    collect.append(url)
    collect.append(content['image'])
    collect.append(content['title'])
    collect.append(content['author'])
    collect.append(content['annotation'].get_text(strip=True) if content['annotation'] != '' else '')
    return collect

# Загрузка списка URL из файла
urls_df = pd.read_csv('book_links(1).csv')

# Открытие файла для записи результатов
with open('books.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, escapechar='\\', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['page_url', 'image_url', 'author', 'title', 'annotation'])

    for index, row in urls_df.iterrows():
        book_data = scrape_book_data(row['book_link'])
        writer.writerow(book_data)
        print(f"Информация о книге: {row['book_link']} записана в файл books.csv")

